# ALGORITMOS BASADOS EN SERIES TEMPORALES

## 5) Modelo SIMPLE - Promedio de Venta Simple (PVS))

Por lo relevado con los compradores es la forma en la que hoy compran. Toman la venta de los últimos 30 dias, la ven separada en 2 quincenas. Aplican criterio en base a los quiebres que haya tenido durante los últimos 30 dias. Exluye los datos de quiebre para calcular el promedio. Si tuvo oferta, le baja un poco la demanda. Si va a tener oferta la sube un poco. Toda esa información futura no está en el sistema.

1) **Preprocesamiento**
* Convertir la columna Fecha a formato datetime (si aún no lo está).
* Seleccionar las columnas necesarias y ordenar los datos cronológicamente.

2) **Agrupación y Resampleo**
* Agrupar los datos por Codigo_Articulo y Sucursal.
* Resamplear las ventas a frecuencia diaria para obtener la suma de Unidades vendidas por día.

3) **Cálculo del Pronóstico**
* Para cada grupo (artículo y sucursal), calcular la media diaria de los últimos N días (por ejemplo, 30 días).
* Multiplicar esta media por el número de días de la ventana de reposición (30 o 45 días) para obtener la cantidad a solicitar.

4) **Salida y Exportación**

* Generar un DataFrame con columnas: Codigo_Articulo, Sucursal y Cantidad (total a pedir).
* Exportar este DataFrame a un archivo CSV para su uso en el siguiente proceso.


In [21]:
import itertools
from datetime import datetime
import pandas as pd
import numpy as np
import statsmodels.api as sm
from random import gauss
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from random import random
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing, Holt

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight') 

# Mostrar el DataFrame resultante
import ace_tools_open as tools

# Evitar Mensajes Molestos
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category= FutureWarning)

In [ ]:
# ELEGIR el PROVEEDOR
proveedor = 20
label = '20_Molinos'

In [13]:
# ELEGIR el PROVEEDOR
proveedor = 140
label = '140_Unilever'

In [14]:


# Se puede leer direto de la BD de SQL o desde un archivo CSV local previamente

# Cargar Datos
data = pd.read_csv(f'data/{label}.csv')
data.head()

# Adecuar Tipos de Datos
data['Sucursal']= data['Sucursal'].astype(int)
data['Familia']= data['Familia'].astype(int)
data['Rubro']= data['Rubro'].astype(int)
data['SubRubro']= data['SubRubro'].astype(int)
data['Clasificacion']= data['Clasificacion'].astype(int)
data['Codigo_Articulo']= data['Codigo_Articulo'].astype(int)
data['Fecha'] = pd.to_datetime(data['Fecha'])  # Convertir a formato datetime si aún no lo está
data.sort_values(by='Fecha', ascending=True)  # Ordenar por fecha de menor a mayor

# Crear una nueva columna con el mes y el año para análisis temporal
data['Año-Mes'] = data['Fecha'].dt.to_period('M')

# Crear una nueva columna con el formato Año-Semana (AAAA-WW) a partir de la columna Fecha
data['Año-Semana'] = data['Fecha'].dt.strftime('%Y-%W')

# Confirmar que la columna sigue siendo un campo datetime
#print(data.dtypes)

#data = data.sort_values(by='Fecha', ascending=True)  # Ordenar en orden ascendente (del más antiguo al más reciente)
#data = data.reset_index()

# Recortar Cantidad de Datos ULTIMO AÑO COMPLETO
df = data[data['Fecha']>='2023-01-01']

In [15]:
# Recortar Cantidad de Datos ULTIMO AÑO COMPLETO
df = data[data['Fecha']>='2024-01-01']

In [16]:
# DEJAR solo las columnas Útiles

# Eliminar Columnas Innecesarias
df_prv = df[['Fecha', 'Codigo_Articulo', 'Sucursal', 'Unidades']]

# Convertir en serie temporal con 'Año-Mes' como índice
#df_prv.set_index('Fecha', inplace=True)

# Verificar después de la transformación
tools.display_dataframe_to_user(name="SET de Datos del Proveedor", dataframe=df_prv)
df_prv.info()


SET de Datos del Proveedor


<class 'pandas.core.frame.DataFrame'>
Index: 895842 entries, 730 to 2201411
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Fecha            895842 non-null  datetime64[ns]
 1   Codigo_Articulo  895842 non-null  int32         
 2   Sucursal         895842 non-null  int32         
 3   Unidades         895842 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int32(2)
memory usage: 27.3 MB


In [22]:
import pandas as pd
from datetime import timedelta

# Supongamos que ya tienes tu DataFrame df_prv con las columnas ['Fecha', 'Codigo_Articulo', 'Sucursal', 'Unidades']
# Asegurarse de que la columna 'Fecha' esté en formato datetime
df_prv['Fecha'] = pd.to_datetime(df_prv['Fecha'])

# Configurar el periodo de pronóstico: 30 o 45 días (cámbialo según sea necesario)
forecast_window = 30  # o 45

# Lista para almacenar los resultados del pronóstico
resultados = []

# Agrupar los datos por 'Codigo_Articulo' y 'Sucursal'
for (codigo, sucursal), grupo in df_prv.groupby(['Codigo_Articulo', 'Sucursal']):
    # Establecer 'Fecha' como índice y ordenar los datos
    grupo = grupo.set_index('Fecha').sort_index()
    
    # Resamplear a diario sumando las ventas
    ventas_diarias = grupo['Unidades'].resample('D').sum().fillna(0)
    
    # Seleccionar un periodo reciente para calcular la media; por ejemplo, los últimos 30 días
    # Si hay menos de 30 días de datos, se utiliza el periodo disponible
    ventas_recientes = ventas_diarias[-30:]
    media_diaria = ventas_recientes.mean()
    
    # Pronosticar la demanda para el periodo de reposición
    pronostico = media_diaria * forecast_window
    
    resultados.append({
        'Codigo_Articulo': codigo,
        'Sucursal': sucursal,
        'Cantidad': round(pronostico, 2)
    })

# Crear el DataFrame de pronósticos
df_pronostico = pd.DataFrame(resultados)

# Exportar el resultado a un CSV para su posterior procesamiento
df_pronostico.to_csv('Solicitudes_Compra.csv', index=False)

# Mostrar los primeros registros del pronóstico
tools.display_dataframe_to_user(name="Pronostico de Compras", dataframe=df_pronostico)


C:\Users\eduar\AppData\Local\Temp\ipykernel_9020\4247699703.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prv['Fecha'] = pd.to_datetime(df_prv['Fecha'])


Pronostico de Compras


### VER DATOS COMPARATIVO y DESVIOS

In [19]:
# Asegurarse de que 'Fecha' sea de tipo datetime
df_prv['Fecha'] = pd.to_datetime(df_prv['Fecha'])

# Configurar la ventana de pronóstico (puede ser 30 o 45 días)
forecast_window = 30

# Determinar la fecha máxima disponible y definir una fecha de corte para separar entrenamiento y validación
max_date = df_prv['Fecha'].max()
cutoff_date = max_date - timedelta(days=forecast_window)

resultados_validacion = []

# Agrupar los datos por 'Codigo_Articulo' y 'Sucursal'
for (codigo, sucursal), grupo in df_prv.groupby(['Codigo_Articulo', 'Sucursal']):
    # Establecer 'Fecha' como índice y ordenar cronológicamente
    grupo = grupo.set_index('Fecha').sort_index()
    
    # Datos de entrenamiento: hasta la fecha de corte
    datos_entrenamiento = grupo.loc[:cutoff_date]
    
    # Verificar que existan suficientes datos para calcular el promedio (por ejemplo, al menos 30 días)
    if len(datos_entrenamiento) < 30:
        continue  # O vemos como manejarlo de otra forma
    
    # Resamplear a ventas diarias en el conjunto de entrenamiento
    ventas_diarias_entrenamiento = datos_entrenamiento['Unidades'].resample('D').sum().fillna(0)
    
    # Calcular la media diaria de los últimos 30 días del periodo de entrenamiento
    ventas_recientes = ventas_diarias_entrenamiento[-30:]
    media_diaria = ventas_recientes.mean()
    
    # Calcular el pronóstico para la ventana definida
    forecast = media_diaria * forecast_window
    
    # Definir el periodo de validación: desde el día siguiente a la fecha de corte hasta completar la ventana
    inicio_validacion = cutoff_date + timedelta(days=1)
    fin_validacion = cutoff_date + timedelta(days=forecast_window)
    
    # Extraer y resumir las ventas reales en el periodo de validación
    ventas_validacion = grupo.loc[inicio_validacion:fin_validacion]['Unidades'].resample('D').sum().fillna(0)
    ventas_reales = ventas_validacion.sum()
    
    # Calcular medidas de error
    error_absoluto = abs(forecast - ventas_reales)
    error_porcentual = (error_absoluto / ventas_reales * 100) if ventas_reales != 0 else None
    
    resultados_validacion.append({
        'Codigo_Articulo': codigo,
        'Sucursal': sucursal,
        'Forecast': round(forecast, 2),
        'Ventas_Reales': round(ventas_reales, 2),
        'Error_Absoluto': round(error_absoluto, 2),
        'Error_Porcentual': round(error_porcentual, 2) if error_porcentual is not None else None
    })

# Crear un DataFrame con los resultados de validación
df_validacion = pd.DataFrame(resultados_validacion)

# Mostrar un resumen de los resultados de validación
tools.display_dataframe_to_user(name="SET de Validación de Datos de Compra", dataframe=df_validacion)

C:\Users\eduar\AppData\Local\Temp\ipykernel_9020\2000152170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prv['Fecha'] = pd.to_datetime(df_prv['Fecha'])


SET de Validación de Datos de Compra


### REVISAR com afecta el error si cambio las VENTANAS a lo largo de la Serie de Datos.

La idea es aplicar una validación cruzada adaptada a series temporales, lo que se conoce como validación "walk-forward" o utilizando divisiones de tiempo (TimeSeriesSplit). Esto te permitirá evaluar tu método de pronóstico en distintos cortes de la serie, simulando el comportamiento en producción a lo largo del tiempo.

Se puede utilizar la función TimeSeriesSplit de scikit-learn para evaluar el forecast basado en el promedio de los últimos 30 días (o el tamaño de ventana que definas). El código se centra en una serie temporal (por ejemplo, para un grupo de artículo y sucursal), pero la idea se puede extender a todos tus grupos.

* La función TimeSeriesSplit recibe como argumento el número de divisiones que queremos realizar. 

* Por defecto, scikit-learn utiliza 5 divisiones. La primera división contiene los datos hasta la fecha 30 días antes del último dato disponible. La segunda división contiene los datos desde el último dato disponible hasta la fecha 29 días antes del último dato disponible, y así sucesivamente.

* La función TimeSeriesSplit devuelve un iterador que genera índices para cada una de las divisiones. La primera división se obtiene con el método split del objeto TimeSeriesSpli

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

# Asegurarse de que 'Fecha' sea de tipo datetime
df_prv['Fecha'] = pd.to_datetime(df_prv['Fecha'])

# Función que calcula el pronóstico usando el promedio de los últimos 'rolling_window' días
def forecast_demand(ventas_series, forecast_window, rolling_window=30):
    # Si la serie es menor al tamaño de la ventana, se usa lo disponible
    window = min(rolling_window, len(ventas_series))
    forecast_value = np.mean(ventas_series[-window:]) * forecast_window
    return forecast_value

# Función para realizar la validación cruzada "walk-forward" sobre una serie temporal
def walk_forward_validation(ventas_diarias, forecast_window, n_splits=5, rolling_window=30):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    errores = []
    ventas = ventas_diarias.values  # Convertir la serie a array
    for train_index, test_index in tscv.split(ventas):
        train_series = ventas[train_index]
        test_series = ventas[test_index]
        # Calcular el pronóstico a partir de los datos de entrenamiento
        pronostico = forecast_demand(train_series, forecast_window, rolling_window)
        # Para comparar, sumamos las ventas reales en los primeros 'forecast_window' días del test
        # Si test_series es menor que la ventana, se usa lo disponible
        ventas_reales = np.sum(test_series[:forecast_window]) if len(test_series) >= forecast_window else np.sum(test_series)
        error = abs(pronostico - ventas_reales)
        errores.append(error)
    return errores

# Ejemplo de uso: seleccionamos un grupo específico (por ejemplo, Codigo_Articulo = 12345 y Sucursal = 1)
codigo = 16293
sucursal = 79
grupo = df_prv[(df_prv['Codigo_Articulo'] == codigo) & (df_prv['Sucursal'] == sucursal)]
grupo = grupo.set_index('Fecha').sort_index()
ventas_diarias = grupo['Unidades'].resample('D').sum().fillna(0)

# Configuramos la ventana de pronóstico y realizamos la validación cruzada
forecast_window = 30  # Puede cambiar a 45 según el caso
errores = walk_forward_validation(ventas_diarias, forecast_window, n_splits=5, rolling_window=30)

print("Errores de validación por cada split:", errores)
print("Error promedio:", np.mean(errores), " Unidades")


Errores de validación por cada split: [446.0, 2161.0, 758.0, 49.0, 5.0]
Error promedio: 683.8  Unidades


C:\Users\eduar\AppData\Local\Temp\ipykernel_9020\3045539676.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prv['Fecha'] = pd.to_datetime(df_prv['Fecha'])
